## Introduction
The objective of the EDA stage is to thoroughly investigate and understand the dataset. We aim to establish how the review sentiment targets are related to the text content, and identify key words or phrases that differentiate between positive and negative sentiment classes in product reviews.

## Contents:
* Preparing the ground
    * Importing Libraries and Datasets
    * Data Check and Preprocessing
* Analysis
    * Example Comments
    * Original Dataset Size Check
    * Corpus Cleaning
    * First Word and Wordcount
    * Wordclouds: Most Common Words
    * BOW: Bag of Words
    * TF-IDF
    * Halfway Conclusion
* Feature Generation
    * Additional Features Generation
    * TF-IDF Feature Generation
    * Saving Metrics to File (.jpg)
* Training
    * Logistic Regression
    * Support Vector Classifier (SVC)
    * XGBoost Classifier (XGBClassifier)
* Conclusion

### Importing Libraries and Datasets

In [1]:
import pandas as pd
import numpy as np
import re
import os
import string
import pickle
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
from datetime import datetime
from wordcloud import WordCloud
from sklearn.feature_extraction.text import (
    ENGLISH_STOP_WORDS,
    CountVectorizer,
    TfidfVectorizer
)
import matplotlib.pyplot as plt
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    roc_auc_score,
)
from io import BytesIO
from PIL import Image
from IPython.display import Image, display

RANDOM_STATE = 42
import nltk

nltk.download("vader_lexicon")
pd.set_option("display.max_columns", 20)
pd.set_option("display.max_rows", 500)


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Morea\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
# url='https://drive.google.com/file/d/1eEtlmdLUTZnyY34g9bL5D3XuWLzSEqBU/view?usp=sharing'
# path ='https://drive.google.com/uc?id=' + url.split('/')[-2]
# df = pd.read_csv(path)

# url="https://drive.google.com/file/d/1x2Tdn1UGhQ6x08yfchUykUJvid257vFY/view?usp=sharing"
# path ='https://drive.google.com/uc?id=' + url.split('/')[-2]
# df2 = pd.read_csv(path)

In [3]:
df = pd.read_csv("D:/Programming/DB's/Positive_negative_reviews_classification/reviews.csv")
df2 = pd.read_csv("D:/Programming/DB's/Positive_negative_reviews_classification/labels.csv")

# Create a mapping dictionary
mapping = {"Positive": 1, "Negative": 0}

# Use the map function to replace values
df2.sentiment = df2.sentiment.map(mapping)

In [4]:
df.shape

(254, 2)

In [5]:
df2.shape

(254, 2)

### Data Check and Preprocessing

#### Duplicates

In [6]:
df.tail(5)

,id,text
249,189,"Thank you, this is beautiful and they loved it."
250,72,Thanks so much. They lookgreat!
251,107,"Emily, \n THANK YOU so much for the new “bric..."
252,271,"Jacqueline, \n I just received the replaceme..."
253,103,Order #(857)982-509708\nI just received my ord...


*  Dataset "reviews" has 254 samples, contain "id" and "text" column

In [7]:
print(df.id.tail(5))
print(df2.id.tail(5))

249    189
250     72
251    107
252    271
253    103
Name: id, dtype: int64
249    189
250     72
251    107
252    271
253    103
Name: id, dtype: int64


* IDs of both CSV files corresponds

In [8]:
df.isnull().sum()

id      0
text    0
dtype: int64

* no NULLs

In [9]:
duplicates = df[df["text"].duplicated(keep=False)]
print(duplicates)

      id                                               text
121  309  I just received my item and it is absolutely M...
177   12  I just received my item and it is absolutely M...


In [10]:
df.text.iloc[121]

'I just received my item and it is absolutely MF Amazing!!!!!! WOW.    \n\n Sent from my iPhone'

In [11]:
print(df2.sentiment.iloc[121])
print(df2.sentiment.iloc[177])

1
1


* we drop the duplicate

In [12]:
df.drop(121, axis=0, inplace=True)
df2.drop(121, axis=0, inplace=True)

In [13]:
df.tail(5)

,id,text
249,189,"Thank you, this is beautiful and they loved it."
250,72,Thanks so much. They lookgreat!
251,107,"Emily, \n THANK YOU so much for the new “bric..."
252,271,"Jacqueline, \n I just received the replaceme..."
253,103,Order #(857)982-509708\nI just received my ord...


* there are no duplicates in text now

## Analysis

### Example Comments

In [14]:
df.text.iloc[0]

'Hi Leah, I ordered this early to give to a co-worker for a Christmas gift. However, I am really impressed with they 3D item came out. I do plan to show it to another co-worker to see what she thinks about the outcome. Thank you reaching out and most of all it was such a quick turnaround.  \n  Great service.  Matthew Mason;\n\n Sent from Yahoo Mail on Android'

In [15]:
df.text.iloc[-1]

'Order #(857)982-509708\nI just received my order!  They are fantastic - even better than I expected.   I ordered 3 different items but only received 1 cleaning cloth.  Can you send me 2 more?  Thank you.  James Young'

In [16]:
df.text.iloc[50]

"Hello,I received my order today. thank-you!! my Daughter will love it!! I'm giving for Christmas gift. thank you again!  I will recommend to friends!!"

In [17]:
df.text.iloc[20]

"I received the Item a few days ago. I was verypleasantly surprised how beautiful the item was. The detail was excellent and the photo looked 3d. I received so many compliments on it. I did make a  huge mistake on it. I put a date of 1974 when it should have been 1973. I can't believe I did that. It was  totally my fault. That takes away something from it for me. There's nothing I can do about it now.  \n  Thank You for everything your and yourteam did. You do excellent work and it's a memory that will last  a lifetime! I will let everyone who I show it to see how your company is the best. Thanks again, Tammy Logan;"

In [18]:
df.text.iloc[70]

'I ordered 5 key chains, I am not happy with.you can’t see the pictures in them .I want to return them.'

* ok, keychains

In [19]:
df.text.iloc[90]

'Hi there!\n\nJust wanted to let you know we did in fact receive your product on time and in beautiful condition.  It is amaaazingly beautifully well done. Thank you so much.\n\nBest Regards,\n\nMichaela & Erik Velasquez'

In [20]:
df.text.iloc[110]

'I just would like to thank you from the bottom of my heart it came out beautiful it really looks like my dad now I can see and be with him everyday thank you very much I will recommend to all my friends and family again thank you\n\n\n  Sent from AOL Mobile Mail\nGet the new AOL app: mail.mobile.aol.com'

* should remove "Sent from...."

In [21]:
df.text.iloc[130]

"Adam, \n\n The towers we gave at Christmas gifts or warmly and passionately received. They loved them as my wife did hers. We wrote a wonderful review but we have not seen it on your website. We would be most happy to do it again if you would send us a link or instructions. Thank you again! \n\n Brendan \n\n Sent from my iPhone\n\nBrendan Lillard \n\n Arthur J. Gallagher & Co.\n\n12 E Central | Port Jefferybury, LA 74354\n\nMobile: 992-584-3185\n\nalexandria17@yahoo.com\n\njames81@yahoo.com\n\n \n\nThe intent of this analysis is to provide you with general information regarding the status of, and/or potential concerns related to, your current employee benefits issue. It does not necessarily fully address all your specific issues. It should not be construed as, nor is it intended to provide, legal advice. Questions regarding specific issues should be addressed by the client's general counsel or an attorney who specializes in this practice area. \n\n \n\nThis e-mail and any files transm

* looks like the shop is selling some custom artwork like statuette and hand-made keychains...

In [22]:
df.text.iloc[150]

'I just opened the medium heart I ordered\nOrder # (673)305-513556\nI had requested the date to be small print and it bolds out, it’s too large \nI’m not happy with my purchase.\nMr. Austin Ellis \n8335955989\nryan64@hotmail.com'

In [23]:
df.text.iloc[169]

'Hi Kimberly, \n  \n I just picked it up from the front desk today.  \n I was going to send you an email before the day was done, & let you know how Wonderful it looks!! \n  \n We appreciate your time, & energy on this creation. Look forward to doing business with you again. \n  \n We are so happy with how it turned out!! \n  \n Thank you, & your team so much! \n  \n Respectfully, \n Joseph Allen \n  \n  \n \nJoseph Allen\n\nPsychometrist, TMA- Mental Health and Wellness \n \nKaiser Permanente\n\n\nOffice: (763)534-5911 \n  \n  \n \nJoseph Allen\n\nPsychometrist, TMA- Mental Health and Wellness \n \nKaiser Permanente\n\n\nOffice: (763)534-5911 \n  \n \n NOTICE TO RECIPIENT:\nIf you are not the intended recipient of this e-mail, you are prohibited from sharing, copying, or otherwise using or disclosing its contents.\nIf you have received this e-mail in error, please notify the sender immediately by reply e-mail and permanently delete this e-mail and any attachments without reading, forw

In [24]:
# Check if any sample contains "NOTICE TO RECIPIENT"
contains_notice = df["text"].str.contains("NOTICE TO RECIPIENT")

# Filter and print samples containing the notice
samples_with_notice = df[contains_notice]
print(samples_with_notice.text)

170    Hi Kimberly, \n  \n I just picked it up from t...
Name: text, dtype: object


* looks like it's such caution "NOTICE TO RECIPIENT" happens once 

In [25]:
df.text.iloc[190]

'Thank you so much for your service has she enjoyed it so much it was more than we expected and if I have any other gifts I want to buy for myself or my family I would definitely recommend you guys you guys are awesome'

In [26]:
df.text.iloc[209]

"Ann Thank you. I love my product. There is a blemish on it where it looks like a piece of tape that illuminates when put on the base. Its noticable but it may be because of how the cats whiskers had to be done. Wasn't expecting that in such a fine product and at what I paid but it's no big deal I suppose. Just looks odd. Very happy otherwise.  \n  Thomas"

* cat and whiskers well made

In [27]:
df.text.iloc[252]

'Order #(857)982-509708\nI just received my order!  They are fantastic - even better than I expected.   I ordered 3 different items but only received 1 cleaning cloth.  Can you send me 2 more?  Thank you.  James Young'

* maybe need to remove order number and leave only word "order" though?

In [28]:
df.text.iloc[123]

'Hi Brooke, \n  I absolutely love it. It’s for my best friends 60th birthday. I will giving you her on September 14. I want to take a photo with the cube and then submit review. Hope you don’t mind. She’s gonna love it as much as I do.   \n  Stay well.  \n  Dorothy  \n  \n\n Sent from my iPhone'

In [29]:
df.text.iloc[97]

"I know this e-mail address is for support stuff but I just wanted to reach out and say how amazing my order turned out! The picture I used was of my brother who passed away in September and I ordered this for my mom. It is incredible and she is going to love it (and cry a lot.) Thank you so much for this wonderful gift, I can't wait to place my next order for a larger size! \n  Thank you again,  \n  \n  James Simmons"

* the store sells photo frames with custom photos

### Original Dataset Size Check

In [30]:
df.sort_values("id", ascending=False)

,id,text
230,319,Im am beyond devastated ups tracking said they...
52,318,I’m so impressed with your work! I just wish I...
178,317,"Hello,\n \n\n \n This has arrived and it’s bea..."
82,316,Awww thank you all for the delivery updates. I...
223,315,Thank you I loved it my one concern was my mes...
159,314,I'm very happy with the item memory that I had...
206,312,I'm very disappointed in the size and of the h...
225,308,"To ALL this may concern, I am one SATISFIED cu..."
111,307,"\n Message \n Order #245371902556 recibed, sup..."
106,305,Hi Bruce: \n Thank you for remaking the item ...


In [31]:
# Create a range of expected IDs (e.g., from 1 to max_id)
max_id = df["id"].max()  # Assuming 'ID' is a numeric column
expected_ids = set(range(1, max_id + 1))

# Convert the column of unique IDs to a set
actual_ids = set(df["id"])

# Find the missing IDs
missing_ids = expected_ids - actual_ids

# Display the missing IDs as a list
missing_id_list = list(missing_ids)
print("Missing IDs:", sorted(missing_id_list, reverse=1))
print("Number of missing IDs:", len(missing_id_list))

Missing IDs: [313, 311, 310, 309, 306, 301, 300, 291, 290, 282, 279, 278, 273, 268, 262, 258, 252, 249, 247, 242, 234, 231, 230, 228, 221, 198, 190, 186, 171, 162, 160, 159, 158, 157, 153, 144, 127, 125, 123, 98, 97, 96, 94, 90, 87, 83, 81, 79, 77, 71, 70, 66, 58, 54, 52, 50, 46, 45, 41, 21, 17, 16, 13, 11, 9, 2]
Number of missing IDs: 66


66(missed IDs)+253(present IDs)=319 (total)

or

319 * 0.8 = 255 or ≈ 253 (present IDs)
* Dataset we were given represents 80% of the entire dataset owned by the customer as was stated 


In [32]:
print(
    f"Number of Positives: {df2.sentiment.value_counts()[0]} or {100*round(df2.sentiment.value_counts()[0]/len(df2), 2)}%"
)
print(
    f"Number of Negatives: {df2.sentiment.value_counts()[1]} or {100*round(df2.sentiment.value_counts()[1]/len(df2), 2)}%"
)

Number of Positives: 102 or 40.0%
Number of Negatives: 151 or 60.0%


* most of ML algorithms can handle this level of class imbalance without significant issues.

In [33]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets (80% train, 20% test)
train_X, test_X, train_y, test_y = train_test_split(df, df2, test_size=0.2, random_state=RANDOM_STATE)

#### Add features (a few)

* need to run it before we lose such details in preprocessing

### Corpus Cleaning

In [34]:
def clean_text(text):
    text = text.lower()

    # Define the pattern to match "Sent from..." and everything after it
    cleaned_text = re.sub(r"Sent from.*$", "", text)

    # Define the pattern to match "Order" followed by numbers and spaces Use re.sub() to replace the pattern with "order"
    cleaned_text = re.sub(r"Order\s*\(#\d+\)", "order number", text)

    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub("\W", " ", text)  # != a-z, A-Z, and 0-9
    text = re.sub(
        "\s+", " ", text
    )  # s(spaces)== \t \n \r (return carret on the beginning)

    text = re.sub("<.*?>", " ", text)
    text = text.translate(str.maketrans(" ", " ", string.punctuation))

    # Keep ! and ? symbols
    text = re.sub("[^a-zA-Z!?]", " ", text)
    text = re.sub("\n", " ", text)
    text = text.strip(" ")
    return text

In [35]:
df["text"] = df["text"].apply(clean_text)

* text cleaned

### First Word and Wordcount

In [36]:
# Extract the first word from each comment and create a new column
df["first_word"] = df["text"].str.split().str[0]

# Sort the DataFrame based on the 'first_word' column
df_sorted = df.sort_values(by="first_word")

# Count the occurrences of each unique first word
word_counts = df_sorted["first_word"].value_counts()

# Display the sorted DataFrame
print("Sorted DataFrame:")
print(df_sorted)

Sorted DataFrame:
      id                                               text  first_word
131  240  adam the towers we gave at christmas gifts or ...        adam
154  179  afternoon mr melinda first let me say i love t...   afternoon
76    82  alan the gift was for my wife and she loved it...        alan
96   302  alexandria foster i received the heart i order...  alexandria
210  191  ann thank you i love my product there is a ble...         ann
199  217  appreciate your response but because the produ...  appreciate
82   316  awww thank you all for the delivery updates i ...        awww
220   89  bradley that s awesome thank you so much i apo...     bradley
233   59  dear alexandra thank for your quick and warm r...        dear
92    80  emily i just want to say that i took your word...       emily
251  107  emily thank you so much for the new brick item...       emily
231  188  everything is good beautiful piece great job m...  everything
25   184  first and foremost i absolutely love

In [37]:
# First word counts
print("First word Counts:")
word_counts

First word Counts:


i             69
hello         24
hi            21
thank         14
message       10
the            9
good           9
my             8
yes            6
to             5
just           5
we             4
it             4
received       3
order          3
how            2
ordered        2
sent           2
kelly          2
you            2
hey            2
thanks         2
what           2
well           2
emily          2
this           2
susan          1
review         1
support        1
sir            1
recibe         1
very           1
product        1
warry          1
sirs           1
so             1
noah           1
adam           1
mr             1
greetings      1
alan           1
alexandria     1
ann            1
appreciate     1
awww           1
bradley        1
dear           1
everything     1
first          1
great          1
have           1
morning        1
if             1
im             1
img            1
in             1
jacqueline     1
james          1
kind          

* most of comments begins with words like: "I"(most cases), "Hello"\"Hi", "Thank (you)"

These introductory words can provide context to the comment's tone and purpose. For instance, comments that start with "I" might focus on the commenter's own thoughts, while comments that start with "Thank you" are likely expressing gratitude. But still, to understand data we need go deeper.

### Wordclouds: Most Common Words 

In [38]:
# Python program to generate WordCloud
comment_words = ""
stopwords = set(ENGLISH_STOP_WORDS)

# iterate through the csv file
for val in df.text:
    # typecaste each val to string
    val = str(val)

    # split the value
    tokens = val.split()

    # Converts each token into lowercase
    for i in range(len(tokens)):
        tokens[i] = tokens[i].lower()

    comment_words += " ".join(tokens) + " "

wordcloud = WordCloud(
    width=800,
    height=800,
    background_color="white",
    stopwords=stopwords,
    min_font_size=10,
).generate(comment_words)

# plot the WordCloud image
plt.figure(figsize=(4.5, 4.5), facecolor=None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad=0)

plt.show()

* the wordcloud shows that the dataset is tematic and mostly consist of customer comments.

the main message is - "received order, thank (or not)"

### BOW: Bag of Words

In [39]:
# Initialize CountVectorizer with optional parameters (you can adjust them as needed)
count_vectorizer = CountVectorizer(stop_words=list(ENGLISH_STOP_WORDS))

# Fit and transform the text data to create the BoW representation
bow_matrix = count_vectorizer.fit_transform(df.text)

# Get feature names (words)
bow_feature_names = count_vectorizer.get_feature_names_out()

# Create a DataFrame to store BoW results
df_bow = pd.DataFrame(bow_matrix.toarray(), columns=bow_feature_names)

# Calculate word frequencies
word_frequencies = df_bow.sum().sort_values(ascending=False)

# Display the top N keywords based on word frequencies (adjust N as needed)
top_keywords_bow = word_frequencies.head(20)
print("Top Keywords (BoW):")
print(top_keywords_bow)

Top Keywords (BoW):
thank           157
item            143
order           104
received         94
sent             81
just             81
disappointed     66
gift             57
beautiful        56
love             54
ordered          53
picture          53
like             47
iphone           46
thanks           45
happy            41
did              40
know             39
today            38
base             37
dtype: int64


### TF-IDF

In [40]:
# Initialize TfidfVectorizer with optional parameters (you can adjust them as needed)
tfidf_vectorizer = TfidfVectorizer(stop_words=list(ENGLISH_STOP_WORDS))

# Fit and transform the text data to create the TF-IDF representation
tfidf_matrix = tfidf_vectorizer.fit_transform(df.text)

# Get feature names (words)
tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()

# Create a DataFrame to store TF-IDF results
df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_feature_names)

# Calculate TF-IDF scores
tfidf_scores = df_tfidf.mean().sort_values(ascending=False)

# Display the top N keywords based on TF-IDF scores (adjust N as needed)
top_keywords_tfidf = tfidf_scores.head(20)
print("\nTop Keywords (TF-IDF):")
print(top_keywords_tfidf)
type(top_keywords_tfidf)


Top Keywords (TF-IDF):
thank           0.057791
item            0.048705
order           0.040628
received        0.040266
sent            0.038786
beautiful       0.038073
just            0.036558
love            0.031706
iphone          0.028826
disappointed    0.027921
ordered         0.027545
thanks          0.026396
great           0.024782
gift            0.024753
picture         0.024234
today           0.023540
happy           0.021568
review          0.020837
base            0.020830
like            0.020805
dtype: float64


pandas.core.series.Series

### Halfway Conclusion

    Keyword Significance: The keywords with the highest TF-IDF and BOW scores are considered significant within the dataset. In your list, "thank," "item," "order," "received," "sent," and "beautiful" have relatively high TF-IDF and BOW scores, suggesting that they are important terms in the reviews.

    Positive Sentiment: Some of the keywords with high TF-IDF and BOW scores, such as "thank," "beautiful," "love," "great," "happy," and "gift," suggest positive sentiment. These words are likely to appear frequently in positive reviews.

    Negative Sentiment: While most of the top keywords appear to indicate positive sentiment, the presence of "disappointed" suggests negative sentiment. It is possible that "disappointed" is a strong indicator of negative reviews.

    Order-Related Terms: Keywords like "item," "order," "received," "sent," "ordered," and "thanks" are related to the process of ordering and receiving items. These terms may indicate that the dataset contains reviews related to online shopping or purchases.

    Product-Specific Terms: Terms like "picture" and "base" are specific to certain products or items. The presence of this terms may indicate that the dataset contains reviews for a variety of products, including items with pictures on some "base".

    Contextual Insights: The presence of certain terms can provide context for the dataset. For example, "today" and "review" may indicate that some reviews were written on the day of receiving the product or are reviews themselves.

## Feature Generation

In [41]:
# Split the data into training and testing sets (80% train, 20% test)
train_X, test_X, train_y, test_y = train_test_split(df.drop(
    columns=['id','first_word']), df2.drop(columns=['id']), test_size=0.2, random_state=RANDOM_STATE)

### Additional features generation

In [42]:
def feature_generator(df):
    # punctuation count
    df.loc[:, "count_punctuations"] = df["text"].apply(
        lambda x: len([c for c in str(x) if c in string.punctuation])
    )  # including "I'll" as punctuation

    # upper case words count
    df.loc[:, "count_words_upper"] = df["text"].apply(
        lambda x: len([w for w in str(x).split() if w.isupper()])
    )

    # title case words count
    df.loc[:, "count_words_title"] = df["text"].apply(
        lambda x: len([w for w in str(x).split() if w.istitle()])
    )

    df["text"] = df["text"].apply(clean_text)
    sentiment = SentimentIntensityAnalyzer()

    # Initialize empty lists for each sentiment score
    neg_scores = []
    neu_scores = []
    pos_scores = []
    compound_scores = []

    # Loop through the sentences and calculate sentiment scores
    for sentence in df["text"]:
        ss = sentiment.polarity_scores(sentence)
        neg_scores.append(ss["neg"])
        neu_scores.append(ss["neu"])
        pos_scores.append(ss["pos"])
        compound_scores.append(ss["compound"])

    # Add sentiment scores as new columns to the DataFrame
    df["negative"] = neg_scores
    df["neutral"] = neu_scores
    df["positive"] = pos_scores
    df["compound"] = compound_scores
    # Word count in each comment:
    df["word_count"] = df["text"].apply(lambda x: len(x.split()))

    # Unique word count
    df.loc[:, "count_unique_word"] = df["text"].apply(lambda x: len(set(str(x).split())))

    # Letter count
    df.loc[:, "count_letters"] = df["text"].apply(lambda x: len(str(x)))

    # Number of stopwords
    df.loc[:, "count_stopwords"] = df["text"].apply(
        lambda x: len([w for w in str(x).lower().split() if w in ENGLISH_STOP_WORDS])
    )

    # Average length of the words
    df.loc[:, "mean_word_len"] = df["text"].apply(
        lambda x: round(np.mean([len(w) for w in str(x).split()]), 2)
    )

    # Derived features

    # Word count percent in each comment:
    df.loc[:, "word_unique_percent"] = (
            df.loc[:, "count_unique_word"] * 100 / df["word_count"]
    )
    # Punct percent in each comment:
    df.loc[:, "punct_percent"] = df.loc[:, "count_punctuations"] * 100 / df["word_count"]

    # Scale features
    scaled_features = StandardScaler().fit_transform(df.drop(columns=["text"]))
    # ↑↑↑ we need to scale them before we stack them with tf-idf features (which we shouldn't scale at all).

    return scaled_features

### TF-IDF Feature Generation

In [43]:
def tfidf(X,max_features=50,features:str="all"):
    scaled_features=feature_generator(X) 
    
    # Inin vectorizer
    tf_idf = TfidfVectorizer(
        ngram_range=(1, 1),
        max_features=max_features,
    )
    
    # Create TF-IDF features
    tfidf_features = tf_idf.fit_transform(X.text).toarray()
    
    # Stack them vertically
    stacked_array = np.hstack((scaled_features, tfidf_features))
    if features=="tfidf":
        out=tfidf_features
    elif features=='all':
        out=stacked_array
    return out

### Saving Metrics to File (.jpg)

In [44]:
def metrics(text, report, conf_matrix, auc, minutes, seconds):
    output_text = (
        f"{text}\n\n"
        "Classification Report:\n\n"
        + report
        + "\n\n"
        "Confusion Matrix:\n"
        + str(conf_matrix)
        + "\n\n"
        "AUC: "
        + str(int(auc * 100) / 100.0)
        + "\n\nElapsed time:\n"
        + f"{minutes} minutes\n{round(seconds, 3)} seconds"
    )

    # Create a fixed-size plot with the output text
    plt.figure(figsize=(8, 6))  # Specify the desired width and height in inches
    plt.text(0.9, 0.5, output_text, fontsize=12, ha="right", va="center", wrap=True)  # Adjust x-coordinate (0.9) for right alignment
    plt.axis("off")

    # Save the plot as an image
    output_image_path = f"./{text}.jpg"
    plt.savefig(output_image_path, bbox_inches="tight", pad_inches=0, dpi=300)
    plt.close()

    # Display the image in the Jupyter Notebook
    return display(Image(filename=output_image_path, width=400))


## Training

In [45]:
train_X=tfidf(train_X)

In [47]:
test_X=tfidf(test_X)

In [50]:
train_y=train_y.sentiment

In [53]:
from sklearn.model_selection import cross_val_score

model = LogisticRegression(random_state=RANDOM_STATE, max_iter=2200)
# Create a pipeline with feature normalization and Logistic Regression
pipeline = make_pipeline(StandardScaler(with_mean=False, with_std=False), model) 
# Fit the model to the data
pipeline.fit(train_X, train_y)
print(pipeline.predict(test_X))

# Perform cross-validation and get an array of scores
scores = cross_val_score(pipeline, train_X, train_y, cv=3, scoring="roc_auc")

# Print the cross-validation scores
print("Cross-Validation Scores:", scores)

# Calculate the mean score and standard deviation
mean_score = scores.mean()
std_score = scores.std()

print("Mean ROC AUC Score:", mean_score)
print("Standard Deviation of ROC AUC Scores:", std_score)



[1 0 1 1 0 0 0 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 0 1 0 0 1 1 0
 0 1 0 1 1 1 0 0 0 1 0 0 1 1]
Cross-Validation Scores: [0.97612732 0.99450549 0.97161172]
Mean ROC AUC Score: 0.9807481790240411
Standard Deviation of ROC AUC Scores: 0.009901025411515788


In [54]:
scaler_hyperparameters = {
    'standardscaler__with_mean': [False],
    'standardscaler__with_std': [False]
}

classifier_hyperparameters = {
    'logisticregression__C': [0.1, 1, 10, 100]
}

from sklearn.model_selection import RandomizedSearchCV 

param_distributions = [
    {
        **scaler_hyperparameters,
        **classifier_hyperparameters
    }
]

random_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_distributions,
    scoring='roc_auc',  # Use an appropriate scoring metric
    cv=3  # Number of cross-validation folds
)


In [57]:
# Create a "models" folder if it doesn't exist
models_folder = "models"
if not os.path.exists(models_folder):
    os.makedirs(models_folder)

# Specify the model name you want to load
model_name = "LogisticRegression with 14 features."  # Change this to the desired model name

# Load the model from the "models" folder
model_filename = f"models/{model_name}_model.pkl"

with open(model_filename, 'rb') as model_file:
    loaded_model = pickle.load(model_file)
#     print(loaded_model.predict(test_X))
    

ValueError: X has 64 features, but LogisticRegression is expecting 50 features as input.

In [60]:

    
random_search.fit(train_X, train_y)

best_pipeline = random_search.best_estimator_
best_model = best_pipeline.named_steps['logisticregression']
best_params = random_search.best_params_
best_score = random_search.best_score_

model_filename = os.path.join(models_folder, f'{model_name}_model.pkl')
with open(model_filename, 'wb') as model_file:
    pickle.dump(model, model_file)

    
# Load the saved model
model_filename = os.path.join(models_folder, f'{model_name}_model.pkl')
with open(model_filename, 'rb') as model_file:
    loaded_model = pickle.load(model_file)

C:\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


In [ ]:
test_X=pd.DataFrame(tfidf())
# Assuming you have test_X and test_y
y_pred = loaded_model.predict(test_X)

# Calculate classification report, confusion matrix, or any other metrics you need
report = classification_report(test_y, y_pred)
conf_matrix = confusion_matrix(test_y, y_pred)
auc = roc_auc_score(test_y, y_pred_prob)  # You may need to calculate this again if not saved earlier

# Print or use the metrics as needed
print(report)
print(conf_matrix)
print(f'AUC: {auc}')


In [ ]:
import pickle


# Perform cross-validation and get predicted probabilities for the positive class
y_pred_prob = cross_val_predict(pipeline, X, y, cv=3, method="predict_proba")[:, 1]

print(y_pred_prob)
    
#     # Calculate classification report
#     y_pred = (y_pred_prob > 0.5).astype(int)  # Convert probabilities to binary predictions

#     # Classification report
#     report = classification_report(
#         y_true, y_pred, target_names=["Negative:0", "Positive:1"]
#     )

#     # Calculate confusion matrix
#     conf_matrix = confusion_matrix(y_true, y_pred)

#     # Calculate AUC
#     auc = roc_auc_score(y_true, y_pred_prob)

#     # End time
#     end = datetime.now()

#     # Calculate elapsed time
#     elapsed_time = end - start

#     # Extract minutes and seconds
#     minutes, seconds = divmod(elapsed_time.total_seconds(), 60)
    
#     # Metrics
#     metrics(text=text,
#         report=report,
#         conf_matrix=conf_matrix,
#         auc=auc,
#         minutes=minutes,
#         seconds=seconds,
#     )
    
    # Return the trained model
#     return pipeline

# # Train the model and get the trained model
# trained_model = run_models(X,y, pipeline='logreg')

# # Save the trained model to a file using pickle
# with open('trained_model.pkl', 'wb') as model_file:
#     pickle.dump(trained_model, model_file)


In [ ]:
from sklearn.model_selection import cross_val_score

def run_models(X, text,pipeline:str='logreg'):  
    if pipeline == 'logreg':
        model=LogisticRegression(random_state=RANDOM_STATE, max_iter=2200)
    elif pipeline == "svc":
        model=SVC(probability=True, random_state=RANDOM_STATE)
    elif pipeline == "xgb":
        model=XGBClassifier(random_state=RANDOM_STATE)
    
    # Set y
    y = df2.sentiment

    # Create a pipeline with feature normalization and Logistic Regression
    pipeline = make_pipeline(StandardScaler(with_mean=False, with_std=False), model) 
    # ↑↑↑ scaler acts as a "dummy" because if we use it for real it'll broke tf-idf features.
 

    # Start time
    start = datetime.now()

    scores = cross_val_score(pipeline, X, y, cv=num_folds, scoring="roc_auc")
   
    # Print the cross-validation scores
    print("Cross-Validation Scores:", scores)
    
    # Perform cross-validation and get predicted probabilities for the positive class
    y_pred_prob = cross_val_predict(pipeline, X, y, cv=3, method="predict_proba")[:, 1]
    
    # Calculate the mean score and standard deviation
    mean_score = scores.mean()
    std_score = scores.std()

    print("Mean ROC AUC Score:", mean_score)
    print("Standard Deviation of ROC AUC Scores:", std_score)
    


    # Actual class labels for cross-validation
    y_true = y

    # Calculate classification report
    y_pred = (y_pred_prob > 0.5).astype(int)  # Convert probabilities to binary predictions

    # Classification report
    report = classification_report(
        y_true, y_pred, target_names=["Negative:0", "Positive:1"]
    )

    # Calculate confusion matrix
    conf_matrix = confusion_matrix(y_true, y_pred)

    # Calculate AUC
    auc = roc_auc_score(y_true, y_pred_prob)

    # End time
    end = datetime.now()

    # Calculate elapsed time
    elapsed_time = end - start

    # Extract minutes and seconds
    minutes, seconds = divmod(elapsed_time.total_seconds(), 60)
    # Metrics
    metrics(text=text,
        report=report,
        conf_matrix=conf_matrix,
        auc=auc,
        minutes=minutes,
        seconds=seconds,
    )
    return model

In [ ]:
X_list=[
    df.drop(columns=["text"]),
            tfidf(max_features=50, features='tfidf'),
            tfidf(max_features=50, features='all'),
            tfidf(max_features=100, features='tfidf'),
            tfidf(max_features=100, features='all')
            ]

### Logistic Regression

In [ ]:

    
logreg_text=[
    "LogisticRegression with 14 features.",
    "LogisticRegression with only 50 TF-IDF features.",
    "LogisticRegression with 50 TF-IDF + 14 Additional features.",
    "LogisticRegression with only 100 TF-IDF features.",
    "LogisticRegression with 100 TF-IDF + 14 Additional features."
]

# Init an empty dictionary to store trained models
trained_models = {}

for i in range(len(X_list)):
    print(logreg_text[i])
    print('features.shape:',X_list[i].shape)
    # Train the model and store it in the dictionary
    trained_models[logreg_text[i]] = run_models(X_list[i], logreg_text[i], pipeline='logreg')
#     run_models(X_list[i],logreg_text[i], pipeline='logreg')
    
# Save the trained models to separate pickle files in the "models" folder
for model_name, model in trained_models.items():
    model_filename = os.path.join(models_folder, f'{model_name}_model.pkl')
    with open(model_filename, 'wb') as model_file:
        pickle.dump(model, model_file)

### Support Vector Classifier (SVC)

In [ ]:
svc_text=[
    "SVC with 14 features.",
    "SVC with only 50 TF-IDF features.",
    "SVC with 50 TF-IDF + 14 Additional features.",
    "SVC with only 100 TF-IDF features.",
    "SVC with 100 TF-IDF + 14 Additional features."
]
for i in range(len(X_list)):
    print(svc_text[i])
    print('features.shape:',X_list[i].shape)
    run_models(X_list[i],svc_text[i], pipeline='svc')

### XGBoost Classifier (XGBClassifier)

In [ ]:
xgb_text=[
    "XGBoost with 14 features.",
    "XGBoost with only 50 TF-IDF features.",
    "XGBoost with 50 TF-IDF + 14 Additional features.",
    "XGBoost with only 100 TF-IDF features.",
    "XGBoost with 100 TF-IDF + 14 Additional features."
]
for i in range(len(X_list)):
    print(xgb_text[i])
    print('features.shape:',X_list[i].shape)
    run_models(X_list[i],xgb_text[i], pipeline='xgb')

## Conclusion:

The dataset provided to us is indeed quite small. While it's possible to perform data oversampling on a larger dataset, it won't significantly impact the results. Moreover, our solution should be efficient and capable of running even on lower-tier machines(c)(raspberry pi?). In terms of computational complexity (Big O notation), a larger dataset would lead to increased model complexity.

During the exploratory data analysis (EDA) phase, the training results showed that both simple logistic regression and Support Vector Classifier (SVC) performed well in making predictions. However, XGBoost outperformed the other models. Now, we move on to the next phase.

Generated images with metrics also can be found in the "./EDA" folder

The notebook works in Jupyter and Colabaratory